In [2]:
import numpy as np
import cv2
import torch
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
from torch import nn,optim
import torchvision
import torchvision.transforms.functional as tf
import torchvision.transforms as tfs
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torchvision.models as models
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import random
import time
import copy
import warnings
warnings.filterwarnings("ignore") # 忽略警告
# from tifffile import TIFF
# import tifffile as tiff
try: # for torchvision<0.4
    from torchvision.models.utils import load_state_dict_from_url
except: # for torchvision>=0.4
    from torch.hub import load_state_dict_from_url
from collections import OrderedDict
import math
import torch.utils.checkpoint as cp
from torchvision.models._utils import IntermediateLayerGetter
import json
import matplotlib.pyplot as plt

In [3]:
############ GID数据集 ###############
COLORCLASS_6c = ['Background','Building','Forest','Farmland','Meadow','Water']
COLORMAP_6c = [ [0, 0, 0],[255, 0, 0],[0, 255, 255],[0, 255, 0], [255, 255, 0],[0, 0, 255]]

COLORCLASS_16c = ('Background','Industrial land','Urban residential','Rural residential','Traffic land',
          'Garden land','Arbor forest','Shrub land',
          'Paddy field','Irrigated land','Dry cropland',
          'Natural meadow','Artificial meadow',
          'River','Lake','Pond')
COLORMAP_16c = [[0,0,0],[200,0,0],[250,0,150],[200,150,150],[250,150,150],
            [200,0,200],[150,0,250],[150,150,250],
            [0,200,0],[150,250,0],[150,200,150],
            [250,200,0],[200,200,0],
            [0,0,200],[0,150,200],[0,200,250]]

COLORCLASS_c = ('B_B','Building_Industrial','Building_Urban','Building_Rural','Building_Traffic',
          'Forest_Garden','Forest_Arbor','Forest_Shrub',
          'Farmland_Paddy','Farmland_Irrigated','Farmland_Dry',
          'Meadow_Natural','Meadow_Artificial',
          'Water_River','Water_Lake','Water_Pond')
COLORMAP_c = [[0,0,0],[0,0,179],[0,51,255],[0,153,255],[26,255,230],
            [77,255,179],[0,128,255],[83,129,53],
            [204,255,51],[255,153,0],[255,26,0],
            [179,255,77],[255,204,0],
            [255,51,0],[230,0,0],[179,0,0]]

cm2lbl_16 = np.zeros(256**3)
cm2lbl_6 = np.zeros(256**3)
cm2lbl = np.zeros(256**3)

# 枚举的时候i是下标，cm是一个三元组，分别标记了RGB值
for i, cm in enumerate(COLORMAP_16c):
    cm2lbl_16[(cm[0]*256 + cm[1])*256 + cm[2]] = i
for i, cm in enumerate(COLORMAP_6c):
    cm2lbl_6[(cm[0]*256 + cm[1])*256 + cm[2]] = i
for i, cm in enumerate(COLORMAP_c):
    cm2lbl[(cm[0]*256 + cm[1])*256 + cm[2]] = i
    
# 将标签按照RGB值填入对应类别的下标信息
def image2label_16(im):
    data = np.array(im, dtype="int32")
    idx = (data[:,:,0]*256 + data[:,:,1])*256 + data[:,:,2]
    return np.array(cm2lbl_16[idx], dtype="int64")

def image2label_6(im):
    data = np.array(im, dtype="int32")
    idx = (data[:,:,0]*256 + data[:,:,1])*256 + data[:,:,2]
    return np.array(cm2lbl_6[idx], dtype="int64")

def image2label(im):
    data = np.array(im, dtype="int32")
    idx = (data[:,:,0]*256 + data[:,:,1])*256 + data[:,:,2]
    return np.array(cm2lbl[idx], dtype="int64")

## STL

In [1]:
import os
import torch
from PIL import Image
import numpy as np

# 自定义操作
def custom_operation(tensor1, tensor2):
    conditions_and_values = {
        (0, 0): 1,(1, 1): 1,(1, 2): 1,(1, 3): 1,(1, 4): 1,(2, 5): 1,(2, 6): 1,
        (2, 7): 1,(3, 8): 1,(3, 9): 1,(3, 10): 1,(4, 11): 1,(4, 12): 1,
        (5, 13): 1,(5, 14): 1,(5, 15): 1,
    }
    values = torch.zeros_like(tensor1)
    for condition, value in conditions_and_values.items():
        values[(tensor1 == condition[0]) & (tensor2 == condition[1])] = value
    return values
# 文件夹路径
folder_path1 = r"D:\TSL\multimask\result\deeplabv3+\STL\6"
folder_path2 = r"D:\TSL\multimask\result\deeplabv3+\STL\16"
gt_folder_path1 = r"D:\TSL\data\GID10_clip\test\gtCoarse"
gt_folder_path2 = r"D:\TSL\data\GID10_clip\test\gtFine"
output_folder_path1 = r"D:\TSL\multimask\result\deeplabv3+\ESAD_stl" 
output_folder_path2 = r"D:\TSL\multimask\result\deeplabv3+\SAD_stl" 

# 确保输出文件夹存在
os.makedirs(output_folder_path1, exist_ok=True)
os.makedirs(output_folder_path2, exist_ok=True)

# 存储处理结果的列表
total_num_ones = 0  
total_num_pixels = 0
total_intersection = 0
total_union1 = 0
total_union2 = 0

for filename in os.listdir(folder_path1):
    a = filename.split(".")[0]  # 获取文件名中的数字部分
    image_path1 = os.path.join(folder_path1, f"{a}.png")
    image_path2 = os.path.join(folder_path2, f"{a}.png")
    gt_image_path1 = os.path.join(gt_folder_path1, f"{a}.tif")
    gt_image_path2 = os.path.join(gt_folder_path2, f"{a}.tif")
    
    image1 = Image.open(image_path1)
    image1 = image2label_6(image1)
    tensor1 = torch.from_numpy(image1)  # 转换为PyTorch Tensor格式
    
    image2 = Image.open(image_path2)
    image2 = image2label_16(image2)
    tensor2 = torch.from_numpy(image2)  # 转换为PyTorch Tensor格式

    gt_image1 = Image.open(gt_image_path1)
    gt_image1 = image2label_6(gt_image1)
    gt_tensor1 = torch.from_numpy(gt_image1)  # 转换为PyTorch Tensor格式
    
    gt_image2 = Image.open(gt_image_path2)
    gt_image2 = image2label_16(gt_image2)
    gt_tensor2 = torch.from_numpy(gt_image2)  # 转换为PyTorch Tensor格式

    result = custom_operation(tensor1, tensor2)
    result1=(result == 1)
    
    
    # 计算值为1的像素数量
    num_ones = torch.sum(result == 1).item()
    total_num_ones += num_ones
    total_num_pixels += tensor1.numel()
    intersection1 = (tensor1 == gt_tensor1) & (result == 1)
    intersection2 = (tensor2 == gt_tensor2) & (result == 1)
    intersection = (intersection1 & intersection2)
    total_intersection += torch.sum(intersection).item()
    
    # 创建输出图像

    # 生成 intersection1 的图像
    height, width = result.shape
    rgba_image1 = np.zeros((height, width, 4), dtype=np.uint8)
    rgba_image2 = np.zeros((height, width, 4), dtype=np.uint8)
    # 标记交集的像素为透明
    rgba_image1[intersection.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image1[~intersection.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image1 = Image.fromarray(rgba_image1, 'RGBA')
    output_path1 = os.path.join(output_folder_path1, f"{a}_intersection.png")
    transparent_image1.save(output_path1)
    
    rgba_image2[result1.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image2[~result1.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image2 = Image.fromarray(rgba_image2, 'RGBA')
    output_path2 = os.path.join(output_folder_path2, f"{a}_intersection.png")
    transparent_image2.save(output_path2)
    

print(f"Total number of pixels with value 1 across all images: {total_num_ones}")
print(f"Total number of pixels across all images: {total_num_pixels}")

percentage_ones = (total_num_ones / total_num_pixels) * 100
print(f"SAD: {percentage_ones:.2f}%")

intersection_over_union = total_intersection/ total_num_pixels 
print("total_intersection", total_intersection)
print(f"ESAD: {intersection_over_union:.4f}")

intersection_over_union = total_intersection/ total_num_ones 
print(f"两个层次都分类正确/两个语义一致: {intersection_over_union:.4f}")


## MTL

In [2]:
import os
import torch
from PIL import Image
import numpy as np

# 自定义操作
def custom_operation(tensor1, tensor2):
    conditions_and_values = {
        (0, 0): 1,(1, 1): 1,(1, 2): 1,(1, 3): 1,(1, 4): 1,(2, 5): 1,(2, 6): 1,
        (2, 7): 1,(3, 8): 1,(3, 9): 1,(3, 10): 1,(4, 11): 1,(4, 12): 1,
        (5, 13): 1,(5, 14): 1,(5, 15): 1,
    }
    values = torch.zeros_like(tensor1)
    for condition, value in conditions_and_values.items():
        values[(tensor1 == condition[0]) & (tensor2 == condition[1])] = value
    return values
# 文件夹路径
folder_path1 = r"D:\TSL\multimask\result\deeplabv3+\MTL\6"
folder_path2 = r"D:\TSL\multimask\result\deeplabv3+\MTL\16"
gt_folder_path1 = r"D:\TSL\data\GID10_clip\test\gtCoarse"
gt_folder_path2 = r"D:\TSL\data\GID10_clip\test\gtFine"
output_folder_path1 = r"D:\TSL\multimask\result\deeplabv3+\ESAD_mtl" 
output_folder_path2 = r"D:\TSL\multimask\result\deeplabv3+\SAD_mtl" 

# 确保输出文件夹存在
os.makedirs(output_folder_path1, exist_ok=True)
os.makedirs(output_folder_path2, exist_ok=True)

# 存储处理结果的列表
total_num_ones = 0  
total_num_pixels = 0
total_intersection = 0
total_union1 = 0
total_union2 = 0

for filename in os.listdir(folder_path1):
    a = filename.split(".")[0]  # 获取文件名中的数字部分
    image_path1 = os.path.join(folder_path1, f"{a}.png")
    image_path2 = os.path.join(folder_path2, f"{a}.png")
    gt_image_path1 = os.path.join(gt_folder_path1, f"{a}.tif")
    gt_image_path2 = os.path.join(gt_folder_path2, f"{a}.tif")
    
    image1 = Image.open(image_path1)
    image1 = image2label_6(image1)
    tensor1 = torch.from_numpy(image1)  # 转换为PyTorch Tensor格式
    
    image2 = Image.open(image_path2)
    image2 = image2label_16(image2)
    tensor2 = torch.from_numpy(image2)  # 转换为PyTorch Tensor格式

    gt_image1 = Image.open(gt_image_path1)
    gt_image1 = image2label_6(gt_image1)
    gt_tensor1 = torch.from_numpy(gt_image1)  # 转换为PyTorch Tensor格式
    
    gt_image2 = Image.open(gt_image_path2)
    gt_image2 = image2label_16(gt_image2)
    gt_tensor2 = torch.from_numpy(gt_image2)  # 转换为PyTorch Tensor格式

    result = custom_operation(tensor1, tensor2)
    result1=(result == 1)
    
    
    # 计算值为1的像素数量
    num_ones = torch.sum(result == 1).item()
    total_num_ones += num_ones
    total_num_pixels += tensor1.numel()
    intersection1 = (tensor1 == gt_tensor1) & (result == 1)
    intersection2 = (tensor2 == gt_tensor2) & (result == 1)
    intersection = (intersection1 & intersection2)
    total_intersection += torch.sum(intersection).item()
    
    # 创建输出图像

    # 生成 intersection1 的图像
    height, width = result.shape
    rgba_image1 = np.zeros((height, width, 4), dtype=np.uint8)
    rgba_image2 = np.zeros((height, width, 4), dtype=np.uint8)
    # 标记交集的像素为透明
    rgba_image1[intersection.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image1[~intersection.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image1 = Image.fromarray(rgba_image1, 'RGBA')
    output_path1 = os.path.join(output_folder_path1, f"{a}_intersection.png")
    transparent_image1.save(output_path1)
    
    rgba_image2[result1.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image2[~result1.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image2 = Image.fromarray(rgba_image2, 'RGBA')
    output_path2 = os.path.join(output_folder_path2, f"{a}_intersection.png")
    transparent_image2.save(output_path2)
    

print(f"Total number of pixels with value 1 across all images: {total_num_ones}")
print(f"Total number of pixels across all images: {total_num_pixels}")

percentage_ones = (total_num_ones / total_num_pixels) * 100
print(f"SAD: {percentage_ones:.2f}%")

intersection_over_union = total_intersection/ total_num_pixels 
print("total_intersection", total_intersection)
print(f"ESAD: {intersection_over_union:.4f}")

intersection_over_union = total_intersection/ total_num_ones 
print(f"两个层次都分类正确/两个语义一致: {intersection_over_union:.4f}")


In [5]:
#既分类正确又层次一致
import os
import torch
from PIL import Image
import numpy as np

# 自定义操作
def custom_operation(tensor1, tensor2):
    # 定义满足条件的像素值
    conditions_and_values = {
        (0, 0): 1, (1, 1): 1, (1, 2): 1, (1, 3): 1, (1, 4): 1, (2, 5): 1, (2, 6): 1,
        (2, 7): 1, (3, 8): 1, (3, 9): 1, (3, 10): 1, (4, 11): 1, (4, 12): 1,
        (5, 13): 1, (5, 14): 1, (5, 15): 1,
    }
    values = torch.zeros_like(tensor1)  # 初始化一个和tensor1形状一样的全0张量
    for condition, value in conditions_and_values.items():
        # 设置满足条件的像素值为1
        values[(tensor1 == condition[0]) & (tensor2 == condition[1])] = value
    return values

# 文件夹路径
folder_path1 = r"C:\Home\TSL\multitask\result\GID_CFTMNet\MTL\6"
folder_path2 = r"C:\Home\TSL\multitask\result\GID_CFTMNet\MTL\16"
gt_folder_path1 = r"C:\Home\TSL\data\GID10_clip\test\gtCoarse"
gt_folder_path2 = r"C:\Home\TSL\data\GID10_clip\test\gtFine"
output_folder_path1 = r"C:\Home\TSL\multitask\result\GID_CFTMNet\ESAD_mtl" 
output_folder_path2 = r"C:\Home\TSL\multitask\result\GID_CFTMNet\SAD_mtl" 

# 确保输出文件夹存在
os.makedirs(output_folder_path1, exist_ok=True)
os.makedirs(output_folder_path2, exist_ok=True)

# 存储处理结果的列表
total_num_ones = 0  
total_num_pixels = 0
total_intersection = 0
total_union1 = 0
total_union2 = 0

for filename in os.listdir(folder_path1):
    a = filename.split(".")[0]  # 获取文件名中的数字部分
    image_path1 = os.path.join(folder_path1, f"{a}.png")
    image_path2 = os.path.join(folder_path2, f"{a}.png")
    gt_image_path1 = os.path.join(gt_folder_path1, f"{a}.tif")
    gt_image_path2 = os.path.join(gt_folder_path2, f"{a}.tif")
    
    image1 = Image.open(image_path1)
    image1 = image2label_6(image1)
    tensor1 = torch.from_numpy(image1)  # 转换为PyTorch Tensor格式
    
    image2 = Image.open(image_path2)
    image2 = image2label_16(image2)
    tensor2 = torch.from_numpy(image2)  # 转换为PyTorch Tensor格式

    gt_image1 = Image.open(gt_image_path1)
    gt_image1 = image2label_6(gt_image1)
    gt_tensor1 = torch.from_numpy(gt_image1)  # 转换为PyTorch Tensor格式
    
    gt_image2 = Image.open(gt_image_path2)
    gt_image2 = image2label_16(gt_image2)
    gt_tensor2 = torch.from_numpy(gt_image2)  # 转换为PyTorch Tensor格式

    result = custom_operation(tensor1, tensor2)
    result1=(result == 1)
    
    
    # 计算值为1的像素数量
    num_ones = torch.sum(result == 1).item()
    total_num_ones += num_ones
    total_num_pixels += tensor1.numel()
    intersection1 = (tensor1 == gt_tensor1) & (result == 1)
    intersection2 = (tensor2 == gt_tensor2) & (result == 1)
    intersection = (intersection1 & intersection2)
    total_intersection += torch.sum(intersection).item()
    
    # 创建输出图像

    # 生成 intersection1 的图像
    height, width = result.shape
    rgba_image1 = np.zeros((height, width, 4), dtype=np.uint8)
    rgba_image2 = np.zeros((height, width, 4), dtype=np.uint8)
    # 标记交集的像素为透明
    rgba_image1[intersection.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image1[~intersection.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image1 = Image.fromarray(rgba_image1, 'RGBA')
    output_path1 = os.path.join(output_folder_path1, f"{a}_intersection.png")
    transparent_image1.save(output_path1)
    
    rgba_image2[result1.numpy()] = [0, 0, 0, 0]  # [R, G, B, A] -> 透明
    rgba_image2[~result1.numpy()] = [255, 0, 0, 255]  # 红色
    transparent_image2 = Image.fromarray(rgba_image2, 'RGBA')
    output_path2 = os.path.join(output_folder_path2, f"{a}_intersection.png")
    transparent_image2.save(output_path2)
    

print(f"Total number of pixels with value 1 across all images: {total_num_ones}")
print(f"Total number of pixels across all images: {total_num_pixels}")

percentage_ones = (total_num_ones / total_num_pixels) * 100
print(f"SAD: {percentage_ones:.2f}%")

intersection_over_union = total_intersection/ total_num_pixels 
print("total_intersection", total_intersection)
print(f"ESAD: {intersection_over_union:.4f}")

intersection_over_union = total_intersection/ total_num_ones 
print(f"两个层次都分类正确/两个语义一致: {intersection_over_union:.4f}")


Total number of pixels with value 1 across all images: 398954524
Total number of pixels across all images: 419430400
SAD: 95.12%
total_intersection 350323016
ESAD: 0.8352
两个层次都分类正确/两个语义一致: 0.8781


In [ ]:
### 选取图片
import os
import shutil

# 定义源文件夹路径和目标文件夹路径
source_folders = [r"C:\Home\TSL\multitask\result\GID_CFTMNet\SAD_stl", 
                  r"C:\Home\TSL\multitask\result\GID_CFTMNet\SAD_mtl", 
                  r"C:\Home\TSL\multitask\result\GID_CFTMNet\ESAD_stl", 
                  r"C:\Home\TSL\multitask\result\GID_CFTMNet\ESAD_mtl"]  # 源文件夹列表
target_folders = [r"C:\Users\TSL\Desktop\GID_cftmnet\SAD_stl", 
                  r"C:\Users\TSL\Desktop\GID_cftmnet\SAD_mtl", 
                  r"C:\Users\TSL\Desktop\GID_cftmnet\ESAD_stl", 
                  r"C:\Users\TSL\Desktop\GID_cftmnet\ESAD_mtl"]  # 目标文件夹列表

# 定义要寻找的文件名
file_names = ["13_intersection.png", "191_intersection.png", "269_intersection.png", 
              "96_intersection.png","807_intersection.png","840_intersection.png",
              "2965_intersection.png","3647_intersection.png","99343_intersection.png"]  # 需要根据实际文件的扩展名调整

for source_folder, target_folder in zip(source_folders, target_folders):
    
    # 确保目标文件夹存在，不存在则创建
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
    
    # 遍历文件名列表
    for file_name in file_names:
        
        # 构建完整的源文件路径
        source_file_path = os.path.join(source_folder, file_name)
        
        # 检查文件是否存在
        if os.path.exists(source_file_path):
            # 构建目标文件路径
            target_file_path = os.path.join(target_folder, file_name)
            
            # 复制文件到目标文件夹
            shutil.copy2(source_file_path, target_file_path)
            print(f"文件 {file_name} 已成功复制到 {target_folder}")
        else:
            print(f"文件 {file_name} 在 {source_folder} 中未找到")